In [17]:
def fetch_df():
    import MySQLdb
    import time
    import pandas as pd

    # MySQL connection parameters
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '',
        'db': 'stocks'
    }

    # Establish a MySQL connection
    mydb = MySQLdb.connect(**db_config)
    # Create a cursor object to execute SQL queries

    mycursor = mydb.cursor()

    # Define your SQL query
    sql_query = "SELECT * FROM stocks.portfolio"

    portfolio=pd.read_sql(sql_query,mydb)


    import pandas as pd
    import concurrent.futures
    import pychrome
    from bs4 import BeautifulSoup
    import traceback
    import warnings
    warnings.filterwarnings("ignore")
    import MySQLdb
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time
    from datetime import datetime 
    import requests


    def fetch_html_for_url(url, MF_Name, stock_name,seconds):
        max_retries = 3
        retries = 0

        while retries < max_retries:
            try:
                browser = pychrome.Browser(url="http://127.0.0.1:9222")
                # Create a new tab
                tab = browser.new_tab()

                # Start the tab
                tab.start()

                # Enable page events
                tab.Page.enable()

                # Navigate to the URL
                tab.Page.navigate(url=url, _timeout=10)

                # Wait for page load event
                if retries >0:
                    tab.wait(20)
                else:
                    tab.wait(seconds)

                # Get the root DOM node of the document
                document = tab.DOM.getDocument()

                # Extract the HTML content from the root node
                html_content = tab.DOM.getOuterHTML(nodeId=document['root']['nodeId'])

                # Close the connection to Chrome
                browser.close_tab(tab)
                data=pd.DataFrame(index=range(0,1))
                data['url']=url
                data['MF_Name'] = MF_Name
                data['Stock_name'] = stock_name
                # Check if the HTML content is retrieved successfully
                if 'outerHTML' in html_content:

                    soup = BeautifulSoup(html_content['outerHTML'], 'html.parser')

                    if 'Stock Quotes' in soup.title.get_text():
                        data['check_RNR']='NR'
                    else:
                        data['check_RNR']='R'
    # -----------------------------------------------------stock_id---------------------------------------------------------------
                    if data['check_RNR'][0]=='R':
                        # Iterate over each <script> tag
                        script_tags = soup.find_all('script')
                        for script_tag in script_tags:
                            if script_tag.string:
                                if 'var nseId' in script_tag.string:
                                    data['nse_id'] = script_tag.string.split('var nseId = "')[1].split('";')[0]
                                    data['bse_id'] = script_tag.string.split('var bseId = "')[1].split('";')[0]

        # ----------------------------------------------------- Price ---------------------------------------------------------------

                        if float(data['bse_id'][0])>0.0:
                            # Find the div element with id 'bsecp'
                            div_element = soup.find('div', id='bsecp')

                            if div_element:
                                try:
                                    current_price = float(div_element.get_text().replace(",",""))
                                except:
                                    try:
                                        current_price = float(div_element.attrs.get('data-numberanimate-value').replace(",",""))
                                    except:
                                        current_price = 0.0
                                data['current_price']=current_price
                                data['previous_price'] = float(soup.find('td', class_='bseprvclose').get_text().replace("'","").replace("\n","").replace(",",""))
                            else:
                                data['current_price']=0.0
                                data['previous_price']=0.0

                        else:
                            # Find the div element with id 'nsecp'                    
                            div_element = soup.find('div', id='nsecp')
                            if div_element:
                                current_price = float(div_element.attrs.get('data-numberanimate-value').replace(",",""))
                                data['current_price']=current_price
                                data['previous_price'] = float(soup.find('td', class_='nseprvclose').get_text().replace("'","").replace("\n","").replace(",",""))
                            else:
                                data['current_price']=0.0
                                data['previous_price']=0.0


        # -----------------------------------------------------TechAnalysis---------------------------------------------------------------
                        div_element = soup.find('div', id='techAnalysis')

                        if div_element:
                            try:
                                data['trend'] = div_element['class'][1]
                            except:
                                data['trend']=None
                        else:
                            data['trend']=None
        # -----------------------------------------------------Swot-features---------------------------------------------------------------

                        div_swot_feature = soup.find('div', class_='swot_feature')

                        if div_swot_feature:
                            strong_tags = div_swot_feature.find_all('strong')
                            strong_values = [strong_tag.get_text() for strong_tag in strong_tags]
                            values = [int(s.split('(')[1].replace(')', '')) for s in strong_values]
                            categories = [s.split('(')[0].strip() for s in strong_values]
                            df_strong_values = pd.DataFrame([values], columns=categories)
                            data=pd.concat([data,df_strong_values],axis=1)
                        else:
                            data=pd.concat([data,pd.DataFrame()],axis=1)

                        try:
                            data['Piotroski_Score']=int(soup.find('div', class_=lambda c: c and c.startswith("nof")).get_text())
                        except:
                            data['Piotroski_Score']=0


                    else:
                        print("Data NR")
                        retries += 3 # Increment stop as the stock url is not valid
                    return data


    #--------------------------------------------------------------------------------------------------------------------------------
                else:
                    print("Failed to retrieve HTML content from the tab.")
                    data=pd.DataFrame()
                    data['url']=url
                    retries += 1  # Increment retries since the content retrieval failed
                return data

            except Exception as e:
                print(f"Error fetching HTML for URL {url}: {e}")
                retries += 1  # Increment retries on exception
                print("traceback",traceback.format_exc())
                browser.close_tab(tab)
                if retries >= max_retries:
                    print(f"Max retries reached for URL {url}. Skipping...")
                    browser.close_tab(tab)
                    data=pd.DataFrame(index=range(0,1))
                    data['url']=url
                    data['MF_Name'] = MF_Name
                    data['Stock_name'] = stock_name
                    data['error']=str(e)
                    data['traceback']=traceback.format_exc()
                    return data

        return None, None, None, None, None, None, None

    def fetch(portfolio,seconds):
        start_time = time.time()
        df = pd.DataFrame()
        chunk_size = 5  # Number of URLs to process in each chunk
        urls_chunks = [portfolio[['url', 'MF_Name', 'Stock_name']][i:i+chunk_size] for i in range(0, len(portfolio['url']), chunk_size)]

        for i, chunk_urls in enumerate(urls_chunks, start=1):
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(fetch_html_for_url, chunk_urls['url'], chunk_urls['MF_Name'], chunk_urls['Stock_name'],[seconds]*len(chunk_urls))

            for result in results:
                df = pd.concat([result, df])
            # Display progress dynamically
            print(f"Progress: {i * chunk_size}/{len(portfolio)}", end='\r')    
            time.sleep(1)

        end_time = time.time()
        df['run_time']=(end_time - start_time) / 60
        print("Time took:", (end_time - start_time) / 60)
        return df

    def unfetch(errored,seconds):
        if len(errored)>0:
            start_time = time.time()
            unfetched = pd.DataFrame()
            chunk_size = 1  
            urls_chunks = [errored[['url', 'MF_Name', 'Stock_name']][i:i+chunk_size] for i in range(0, len(errored), chunk_size)]

            for i,chunk_urls in enumerate(urls_chunks, start=1):
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    results = executor.map(fetch_html_for_url, chunk_urls['url'], chunk_urls['MF_Name'], chunk_urls['Stock_name'],[seconds]*len(errored))

                for result in results:
                    unfetched = pd.concat([result, unfetched])
                time.sleep(1)
                print(f"Progress: {i * chunk_size}/{len(errored)}", end='\r') 
            end_time = time.time()
            unfetched['run_time']=(end_time - start_time) / 60
            print("Time took:", (end_time - start_time) / 60)
        return unfetched


    def upload_sql(data, table):
        try:
            # MySQL connection parameters
            db_config = {
                'host': 'localhost',
                'user': 'root',
                'password': 'Hanju@1993',
                'db': 'stocks'
            }

            # Establish a MySQL connection
            mydb = MySQLdb.connect(**db_config)
            mycursor = mydb.cursor()

            # Construct SQL statement dynamically
            columns = ', '.join(data.columns)
            placeholders = ', '.join(['%s'] * len(data.columns))
            sql = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"

            # Execute the SQL statement for each row in the DataFrame
            for index, row in data.iterrows():
                values = tuple(row)
                mycursor.execute(sql, values)

            # Commit the changes and close the cursor and connection
            mydb.commit()
            print(f"All records inserted.")
            mycursor.close()
            mydb.close()
        except Exception as e:
            print(e)
            print(traceback.format_exc())


    df=fetch(portfolio,10)
    try:
        df1=df[(df['check_RNR']=='NR') | (df['traceback'].notna())]
    except:
        try:
            df1=df[df['check_RNR']=='NR']
        except:
            df1=pd.DataFrame()

    if len(df1)>0:
        try:
            try:
                if 'traceback' in unfetched.columns:
                    errored=df[(df['check_RNR']=='NR') | (df['traceback'].notna())].reset_index(drop=True)
                    unfetched=unfetch(errored,20)
                else:
                    errored=df[(df['check_RNR']=='NR')].reset_index(drop=True)
                    unfetched=unfetch(errored,30)                
            except Exception as e:
                print(e,traceback.format_exc())

            try:
                if 'traceback' in unfetched.columns:
                    errored1=unfetched[(unfetched['check_RNR']=='NR') | (unfetched['traceback'].notna())].reset_index(drop=True)
                    unfetched1=unfetch(errored1,30)
                else:
                    errored1=unfetched[(unfetched['check_RNR']=='NR')].reset_index(drop=True)
                    unfetched1=unfetch(errored1,30)
            except Exception as e:
                print(e,traceback.format_exc())


            if 'traceback' in df.columns:
                df_result=df[(df['check_RNR']!='NR') & (df['traceback'].isna())].reset_index(drop=True)
            else:
                df_result=df[(df['check_RNR']!='NR')].reset_index(drop=True) 

            if 'traceback' in unfetched.columns:
                unfetch_result1=unfetched[(unfetched['check_RNR']!='NR') & (unfetched['traceback'].isna())].reset_index(drop=True)
            else:
                unfetch_result1=unfetched[(unfetched['check_RNR']!='NR')].reset_index(drop=True)

            if 'traceback' in unfetched1.columns:
                unfetch_result2=unfetched1[(unfetched1['check_RNR']!='NR') & (unfetched1['traceback'].isna())].reset_index(drop=True)
            else:
                unfetch_result2=unfetched1[(unfetched1['check_RNR']!='NR')].reset_index(drop=True)
        except Exception as e:
                print(e,traceback.format_exc())

        try:
            if len(unfetch_result1)==0:
                unfetch_result1-pd.DataFrame()
            if len(unfetch_result2)==0:
                unfetch_result2 = pd.DataFrame()

            final_result=pd.concat([df_result,unfetch_result1,unfetch_result2]).reset_index(drop=True)
            final_result[final_result.select_dtypes('float').columns]=final_result.select_dtypes('float').fillna(0.0)
            final_result[final_result.select_dtypes('object').columns]=final_result.select_dtypes('object').fillna('None')
            final_result['loaded_time']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            final_result['loaded_time']=pd.to_datetime(final_result['loaded_time'])
            upload_sql(final_result,'MF_stocks')
            upload_sql(final_result[['url','MF_Name','Stock_name','nse_id','bse_id']].drop_duplicates().reset_index(drop=True),'stock_ids')
        except Exception as e:
                print(e,traceback.format_exc())
    else:
        try:
            if 'traceback' not in df.columns:
                df['traceback']=None

            if 'error' not in df.columns:
                df['error']=None

            df[df.select_dtypes('float').columns]=df.select_dtypes('float').fillna(0.0)
            df[df.select_dtypes('object').columns]=df.select_dtypes('object').fillna('None')
            df['loaded_time']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df['loaded_time']=pd.to_datetime(df['loaded_time'])
            # List the columns in the desired order
            desired_order = ['error', 'traceback','run_time','loaded_time']

            # Move the specified columns to the desired position
            new_columns = [col for col in df.columns if col not in desired_order]+desired_order

            # Reindex the DataFrame with the new column order
            df=df.reindex(columns=new_columns)
            upload_sql(df,'MF_stocks')
            upload_sql(df[['url','MF_Name','Stock_name','nse_id','bse_id']].drop_duplicates().reset_index(drop=True),'stock_ids')
        except Exception as e:
            print(e,traceback.format_exc())



    import MySQLdb
    import time
    import pandas as pd

    # MySQL connection parameters
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'Hanju@1993',
        'db': 'stocks'
    }

    # Establish a MySQL connection
    mydb = MySQLdb.connect(**db_config)
    # Create a cursor object to execute SQL queries

    mycursor = mydb.cursor()

    # Define your SQL query
    sql_query = """SELECT p.*,s.current_price,s.previous_price FROM 
                stocks.portfolio as p

                join stocks.MF_stocks as s
                on s.MF_Name=p.MF_Name and s.url=p.url
                where s.loaded_time=(select max(loaded_time) from stocks.MF_stocks)"""

    latest_data=pd.read_sql(sql_query,mydb)

    def convert_to_float(quantity_str):
        try:
            if 'k' in quantity_str:
                return float(quantity_str.replace(' k', '')) * 1000
            elif 'L' in quantity_str:
                return float(quantity_str.replace(' L', '')) * 100000
            elif 'Cr' in quantity_str:
                return float(quantity_str.replace(' Cr', '')) * 10000000
            elif '-' in quantity_str:
                return float(0.0)    
            else:
                return float(quantity_str)
        except:
            return quantity_str

    # Apply the function to the Quantity column

    latest_data['Quantity_mod']=latest_data['Quantity'].apply(lambda x: convert_to_float(x))

    Total_change=(sum(latest_data['current_price']*latest_data['Quantity_mod'])-sum(latest_data['previous_price']*latest_data['Quantity_mod']))/sum(latest_data['previous_price']*latest_data['Quantity_mod'])*100


    result_df = latest_data.groupby('MF_Name').apply(lambda x: (sum(x['current_price'] * x['Quantity_mod'])-sum(x['previous_price'] * x['Quantity_mod']))/sum(x['previous_price'] * x['Quantity_mod'])*100).reset_index(name='percent_change_MF').astype({'percent_change_MF': 'float64'})
    result_df['percent_change_MF'] = result_df['percent_change_MF'].apply(lambda x: '{:.2f}'.format(x))
    result_df


    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    import pandas as pd
    from datetime import datetime

    def send_email(sender_email, sender_password, receiver_email, subject, body, df):
        # Convert DataFrame to HTML
        html_content = df.to_html(index=False)

        # Create a MIME multipart message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject

        msg.attach(MIMEText(body, 'plain'))
        # Attach HTML content as the email body
        msg.attach(MIMEText(html_content, 'html'))

        # Connect to Gmail's SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()

        # Login to the Gmail account
        server.login(sender_email, sender_password)

        # Send email
        server.sendmail(sender_email, receiver_email, msg.as_string())

        # Quit the server
        server.quit()


    # Example usage
    if __name__ == "__main__":
        # Gmail credentials
        sender_email = ''  # Replace with your Gmail address
        sender_password = ''  # Replace with your Gmail password
        receiver_email = ''  # Replace with recipient's Gmail address

        # Email details
        subject = f'Portfolio_Changes_{datetime.now().strftime("%d-%m-%Y")}'
        body=f'Total Portfolio Change is {Total_change}'

        # Send email
        try:
            send_email(sender_email, sender_password, receiver_email, subject,body, result_df)
            print("Mail Sent Succesfully")
        except:
            Print("Unscuccessful")
    return df,df1


In [ ]:
import schedule,time

schedule.clear()
# Schedule the task to run every weekday at 17:32
schedule.every().monday.at("17:32").do(fetch_df)
schedule.every().tuesday.at("17:32").do(fetch_df)
schedule.every().wednesday.at("17:32").do(fetch_df)
schedule.every().thursday.at("17:32").do(fetch_df)
schedule.every().friday.at("17:32").do(fetch_df)

while True:
    # Check if there are any pending jobs to run
    schedule.run_pending()
    time.sleep(60)


Exception in thread Thread-6127:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Exception in thread Thread-6129:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 9

Exception in thread Thread-6143:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6163:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6173:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6185:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


...

Exception in thread Thread-6545:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6555:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6565:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Exception in thread Thread-6573:
Traceback (most recent call last):
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\hanis\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hanis\anaconda3\lib\site-packages\pychrome\tab.py", line 122, in _recv_loop
    message = json.loads(message_json)
  File "C:\Users\hanis\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\hanis\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


Time took: 19.01652467250824
All records inserted.
All records inserted.
Mail Sent Succesfully
